In [1]:
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

import scipy.io.wavfile as wv
import scipy.signal as sig
import wave
from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir
if osDir == winDir:
    storageFolder = 'E:/SAA_DATA/storedData/'
else:
    storageFolder = '/Users/etto/Desktop/storedData/'

Using TensorFlow backend.


In [2]:
def retrieveTestDataMFCC(storageName, keyName):
    filename = storageFolder + storageName + '_MFCC.hd5'
    allMfccsDf = pd.read_hdf(path_or_buf=filename, key='mfccs_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return allMfccsDf.values, classesDf.values

def retrieveTestData(storageName, NFFT, keyName):
    filename = storageFolder + storageName + '_NFFT{:d}.hd5'.format(NFFT)
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf.values, classesDf.values

In [3]:
testFileName = 'testData_G428'
keyname = 'mic4'
NFFT = 1024
testSpecs, testClasses = retrieveTestData(testFileName, NFFT, keyname)
print(np.shape(testSpecs))

(11976, 513)


In [4]:
testFileName = 'testData_G428'
keyname = 'mic4'
NFFT = 2048
testSpecs, testClasses = retrieveTestData(testFileName, NFFT, keyname)
print(np.shape(testSpecs))

(5913, 1025)


In [5]:
testFileName = 'testData_G428'
keyname = 'mic4'
NFFT = 4096
testSpecs, testClasses = retrieveTestData(testFileName, NFFT, keyname)
print(np.shape(testSpecs))

(2886, 2049)


In [8]:
testFileName = 'testData_G428'
keyname = 'mic4'
testSpecs, testClasses = retrieveTestDataMFCC(testFileName, keyname)
print(np.shape(testSpecs))

(12235, 13)


In [ ]:
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs, NFFT)